In [2]:
import json
import re

def parse_txt_to_json_with_index(file_path, output_json_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
    
    blocks = content.split('----------')
    result_list = []
    counter = 1  # dùng để tạo Index dạng 5 chữ số

    for block in blocks:
        block = block.strip()
        if not block:
            continue

        data = {}

        # Lấy EVALUATING SUMMARY số thứ tự
        eval_match = re.search(r'EVALUATING SUMMARY\s*(\d+)', block)
        if eval_match:
            data['Index'] = f"{int(eval_match.group(1)):05d}"
        else:
            data['Index'] = f"{counter:05d}"

        # Lấy Article
        article_match = re.search(r'Article:\s*(.*?)(?=Summary:|$)', block, re.DOTALL)
        if article_match:
            data['Article'] = article_match.group(1).strip()

        # Lấy Summary
        summary_match = re.search(r'Summary:\s*(.*?)(?=Result:|$)', block, re.DOTALL)
        if summary_match:
            data['Summary'] = summary_match.group(1).strip()

        # Lấy Result / Raw result
        raw_result_match = re.search(r'Raw result: ```json\s*(\{.*?\})\s*```', block, re.DOTALL)
        if not raw_result_match:
            raw_result_match = re.search(r'Result:\s*(\{.*\})', block, re.DOTALL)
        if raw_result_match:
            try:
                result_dict = eval(raw_result_match.group(1))
                data['Result'] = result_dict
            except:
                data['Result'] = raw_result_match.group(1)

        result_list.append(data)
        counter += 1

    # Xuất ra file JSON
    with open(output_json_path, 'w', encoding='utf-8') as f_out:
        json.dump(result_list, f_out, ensure_ascii=False, indent=4)

    print(f"Đã xuất kết quả ra file: {output_json_path}")

# Sử dụng
input_file = "data.txt"
output_file = "output.json"
parse_txt_to_json_with_index(input_file, output_file)


Đã xuất kết quả ra file: output.json
